In [1]:
import pandas as pd

In [2]:
import numpy as np

In [6]:
import requests

In [7]:
pip install bs4

     |████████████████████████████████| 122kB 6.8MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [11]:
from bs4 import BeautifulSoup

Scraping Data from Wikipedia Page

In [12]:
source = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(source).text
parsed = BeautifulSoup(response, 'html.parser')

# Removing header row from the table
parsed.tbody.tr.extract()

temp = []
for tr in parsed.tbody.find_all('tr'):    
    temp.append([td.get_text().strip() for td in tr.find_all('td')])
    #removing all Not assigned items
    if temp[-1][1]=='Not assigned':
        temp.pop()

Dataframe Creation and Cleaning

In [13]:
df=pd.DataFrame(temp,columns=['PostalCode','Borough','Neighborhood'])

# Grouping rows with the same postal code
df=df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

# Cleaning up Not assigned neighborhoods
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
        
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [14]:
df.shape

(103, 3)